In [136]:
import numpy as np
import pandas as pd
from sklearn import preprocessing, model_selection
import tensorflow as tf

ImportError: cannot import name 'OneHotEncoder' from 'sklearn' (/home/mrguss/.local/lib/python3.10/site-packages/sklearn/__init__.py)

In [132]:
data = pd.read_csv("pok.csv")
data.tail()
#data.dropna()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True
799,721,Volcanion,Fire,Water,600,80,110,120,130,90,70,6,True


In [181]:
enc = preprocessing.OneHotEncoder(handle_unknown='ignore')
enc.fit(np.array(data[["Type 1", "Type 2"]]))
cat = enc.categories_[0]
def enc(arr):
    ret = np.zeros((len(arr),len(cat)))
    for elem in range(len(arr)):
        if pd.isna(arr[elem]):
            continue
        ret[elem][cat.tolist().index(arr[elem])] = 1
    return ret

In [166]:
cat

array(['Bug', 'Dark', 'Dragon', 'Electric', 'Fairy', 'Fighting', 'Fire',
       'Flying', 'Ghost', 'Grass', 'Ground', 'Ice', 'Normal', 'Poison',
       'Psychic', 'Rock', 'Steel', 'Water'], dtype=object)

In [185]:
y=enc(data["Type 2"])+enc(data["Type 1"])

In [134]:
x = np.array(data.drop(["#", "Name", "Type 1", "Type 2", "Generation", "Legendary"], 1))
y = np.array(data["Star type"], dtype='float')
y.shape = (len(y),1)
x = preprocessing.scale(x)
x

/tmp/ipykernel_9632/269059071.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  x = np.array(data.drop(["Total", "HP", "Attack", "Defense", "Sp. Atk", "Sp. Def","Speed"], 1))


KeyError: 'Star type'

In [91]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size=0.3)

In [52]:
x_train

array([[ 2.63700e+03,  7.30000e-04,  1.27000e-01,  1.72200e+01],
       [ 2.99400e+03,  7.20000e-03,  2.80000e-01,  1.34500e+01],
       [ 2.87100e+03,  7.20000e-04,  1.20000e-01,  1.94300e+01],
       [ 1.41000e+04,  6.70000e-04,  8.90000e-03,  1.21700e+01],
       [ 1.87340e+04,  2.24780e+05,  4.60000e+01, -7.45000e+00],
       [ 6.38000e+03,  1.35000e+00,  9.80000e-01,  2.93000e+00],
       [ 3.14200e+03,  1.32000e-03,  2.58000e-01,  1.41200e+01],
       [ 2.83100e+03,  2.31000e-04,  9.15000e-02,  1.62100e+01],
       [ 1.28930e+04,  1.84000e+05,  3.60000e+01, -6.34000e+00],
       [ 3.54100e+03,  1.30000e-03,  2.56000e-01,  1.43300e+01],
       [ 2.50000e+04,  5.60000e-02,  8.40000e-03,  1.05800e+01],
       [ 3.53100e+03,  9.30000e-04,  9.76000e-02,  1.99400e+01],
       [ 1.71400e+04,  8.83000e+02,  5.65300e+00, -2.64000e+00],
       [ 3.27000e+03,  1.50000e+05,  8.80000e+01, -6.02000e+00],
       [ 3.69200e+03,  3.67000e-03,  4.70000e-01,  1.08000e+01],
       [ 2.41450e+04,  3.

In [48]:
y_train

array([[2.],
       [0.],
       [4.],
       [0.],
       [3.],
       [5.],
       [0.],
       [3.],
       [5.],
       [5.],
       [1.],
       [3.],
       [4.],
       [3.],
       [0.],
       [2.],
       [0.],
       [2.],
       [5.],
       [4.],
       [5.],
       [5.],
       [1.],
       [3.],
       [4.],
       [2.],
       [3.],
       [2.],
       [4.],
       [4.],
       [4.],
       [1.],
       [5.],
       [4.],
       [4.],
       [3.],
       [5.],
       [4.],
       [1.],
       [0.],
       [2.],
       [4.],
       [5.],
       [0.],
       [0.],
       [3.],
       [1.],
       [0.],
       [5.],
       [3.],
       [3.],
       [3.],
       [4.],
       [5.],
       [1.],
       [0.],
       [5.],
       [4.],
       [5.],
       [2.],
       [5.],
       [5.],
       [5.],
       [5.],
       [5.],
       [2.],
       [4.],
       [3.],
       [2.],
       [2.],
       [4.],
       [2.],
       [5.],
       [3.],
       [0.],
       [1.],
       [0.],

In [127]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(4, activation=tf.nn.relu))
#model.add(tf.keras.layers.Dense(5, activation=tf.nn.relu))

model.add(tf.keras.layers.Dense(6, activation=tf.nn.softmax))

model.compile(optimizer="SGD",
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [130]:
model.fit(x_train, y_train,epochs=1500)

Epoch 1/1500
6/6 [==============================] - 0s 3ms/step - loss: 0.0810 - accuracy: 1.0000
Epoch 2/1500
6/6 [==============================] - 0s 4ms/step - loss: 0.0809 - accuracy: 1.0000
Epoch 3/1500
6/6 [==============================] - 0s 4ms/step - loss: 0.0810 - accuracy: 1.0000
Epoch 4/1500
6/6 [==============================] - 0s 4ms/step - loss: 0.0808 - accuracy: 1.0000
Epoch 5/1500
6/6 [==============================] - 0s 4ms/step - loss: 0.0810 - accuracy: 1.0000
Epoch 6/1500
6/6 [==============================] - 0s 5ms/step - loss: 0.0808 - accuracy: 1.0000
Epoch 7/1500
6/6 [==============================] - 0s 5ms/step - loss: 0.0810 - accuracy: 1.0000
Epoch 8/1500
6/6 [==============================] - 0s 4ms/step - loss: 0.0807 - accuracy: 1.0000
Epoch 9/1500
6/6 [==============================] - 0s 4ms/step - loss: 0.0807 - accuracy: 1.0000
Epoch 10/1500
6/6 [==============================] - 0s 5ms/step - loss: 0.0808 - accuracy: 1.0000
Epoch 11/1500
6/6 [

KeyboardInterrupt: 

In [129]:
score,acc = model.evaluate(x_test,y_test)

3/3 [==============================] - 0s 6ms/step - loss: 0.1344 - accuracy: 0.9722
